### 노래 데이터

In [1]:
import numpy as np
import pandas as pd

from typing import Optional
import torch
import transformers
from transformers import AutoModelWithLMHead, PreTrainedTokenizerFast, GPT2LMHeadModel
import fastai
from fastai.text.all import *
import re

c:\Users\kimso\anaconda3\envs\test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
lyrics1 = pd.read_csv("./dataset/label_result_song_short.csv")
lyrics2 = pd.read_csv("./dataset/translated_lyrics.csv")

In [3]:
lyrics = lyrics2.merge(lyrics1, on=["index", "id", "title", "singer", "genres", "lyrics"])

In [4]:
# 각 id에 대해 sentiment_label의 빈도를 계산하여, 가장 많이 나온 2개의 감정을 새로운 칼럼으로 추가

def get_top_two_sentiments(sentiments):
    sentiment_counts = sentiments.value_counts()
    # 가장 많은 2개 감정을 반환
    top_two = sentiment_counts.head(2).index.tolist()
    # 2개 미만일 경우 빈 값 처리
    return top_two + [None] * (2 - len(top_two))

# groupby와 agg를 활용해 변환
lyrics_final = lyrics.groupby("id").agg({
    "title": "first",   # 첫 번째 값 유지
    "singer": "first",  # 첫 번째 값 유지
    "genres": "first",  # 첫 번째 값 유지
    "lyrics": "first",  # 첫 번째 값 유지
    "translated_lyrics": list,  # 리스트로 묶음
    "sentiment_label": lambda x: get_top_two_sentiments(x)  # 빈도가 높은 2개의 감정 추출
}).reset_index()

# sentiment_label에서 두 개의 감정을 각각의 칼럼으로 분리
lyrics_final[['top_sentiment', 'second_sentiment']] = pd.DataFrame(lyrics_final['sentiment_label'].tolist(), index=lyrics_final.index)

# 불필요한 sentiment_label 열 삭제
lyrics_final = lyrics_final.drop(columns=["sentiment_label"])

### 소설 데이터

In [6]:
novel_final = pd.read_csv("./dataset/novel_final.csv")

### fine-tuning

In [7]:
data = novel_final.copy()
data = data[:int(len(data)*0.1)]

In [8]:
from sklearn.preprocessing import LabelEncoder

# 감정 카테고리
emotions = ['상처', '불안', '기쁨', '슬픔', '분노', '당황']

# LabelEncoder 초기화
le = LabelEncoder()
le.fit(emotions) # 감정 카테고리 인코딩

LabelEncoder()

In [9]:
# 감정1과 감정2를 숫자로 변환
data['감정1_encoded'] = le.transform(data['top_sentiment'])
data['감정2_encoded'] = le.transform(data['second_sentiment'])

# 수치형 감정1, 감정2와 텍스트 결합
data['input_text'] = data['감정1_encoded'].astype(str) + " | " + data['감정2_encoded'].astype(str) + " | " + data['text']

data['input_text'].head()

0    2 | 1 | ['01범보다 무서운 곶감', '화자를 처음 만나 이야기를 들으러 왔다고 하자 서슴없이 꺼낸 첫 이야기이다. 화자로서 가장 쉽게 기억해낸 이야기인 셈이다. 설화 앞뒤에 교훈적 해석을 덧붙이고 있음은 화자의 습관화된 태도의 한 모습이기도 하다. 어려서 조모로부터 들었다고 했다.', '그링깨. 사람이 어거지루는 못 살구. 응? 어거지루 안 되능 거여. 사람이 그링깨 뭐이냐 하먼 자연~간 제절루 되야지 어거지루는 못 살어, 사람이.', '그래 옛날, 그 꼭감이라능 게 말여. 사람이 먹잖야 이케? 먹지마는. 그게 참 무성(무서운) 거여.', '애기가 울어. 옛날에. 그래 할머니가 달갸(달래). 그때 호랭이가, 응? 그 집 문앜이 와 섰어 지금. 옛날이는 호랭이가 말두 하구 그랴. 그래 인제 그 할머니가 애기를 달램성 왼갖 소리를 다 햐. ‘호랭이 왔다’구 해두 울구우, ‘괭이 왔다’두 울구, 왼갖 소리를 다 해두 울어.', '그랑깨 꼭감을,', '“아나, 꼭감.”', '“아가, 꼭감.”', '그랑깨 꺄. 안 울어.', '호랭이 생각이? ‘야아, 이거 내가 젤 무선 짐승인디 말여? 나보다 무서웅 게 익구나. 호랭이, 꼭감이, 꼭갬이 참 무석구나....
1    4 | 2 | ['<봄꿩은 제 울음에 저 죽는다>', '그 말과 같아서 사램이 잘못 되머넌 하는 말여.', '지금 강 강사가 말여. 뭘 잘못 돼서 다치게 되머넌…', '아 그, ‘안 했이먼 몰를 긴디 지가 했잉깨 그렇지…’', '그래,', '<봄꿩은 제 울음에 죽는다>', '지가 해놓고 죽는다 그 말이지.', '그래 봄 꿩은 지가 워디 묻혔능가 모르는디 “껄 껄” 항깨루 거깄는 줄 알구 (사람이) 찾아 가능 기여.', '07뱀을 밴 여자', '화자의 체험담이다. 정상적인 임신이 아니고 잘못하여 사람이 생물을 밴 것을 이 지역 사람들은 흔히 ‘농알(어떤 이는 ‘용알’이라고도 함. 의학상으로는 포도상 가태)뱄다’고 말한다. 임신한 여자가 배가 아프다고 하기에 가만히 진단해보니 농알을 밴 듯했다

In [17]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

# KOGPT2 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>') 

model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [18]:
# 데이터셋 클래스 정의
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def encodes(self, x): 
        # Tokenizing input text with padding and truncation
        encodings = self.tokenizer(x, truncation=True, padding='max_length', max_length=128, return_tensors="pt")
        
        # Extracting input_ids and attention_mask
        input_ids = encodings['input_ids']
        attention_mask = encodings['attention_mask']
        
        print(f"Tokenized: {input_ids.shape} - {input_ids}")
        return input_ids.squeeze(0), attention_mask.squeeze(0)

    def decodes(self, x):
        return self.tokenizer.decode(x, skip_special_tokens=True)


In [19]:
print(len(data))

897


In [20]:
# 텍스트 데이터를 리스트 형태로 변환
train = data['input_text'][:int(len(data)*0.9)].tolist()
test = data['input_text'][int(len(data)*0.9):].tolist()

splits = [[0],[1]]

# DataLoader 초기화
tls = TfmdLists([train,test], TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)

batch,seq_len = 2, 64
dls = tls.dataloaders(bs=batch, seq_len=seq_len, num_workers=4, pin_memory=True)
dls.device = torch.device("cuda")

Tokenized: torch.Size([807, 128]) - tensor([[ 9045,   739,   466,  ..., 19561, 12346,   387],
        [ 9130,   739,   466,  ...,  9566,  9181,  6919],
        [ 9130,   739,   466,  ..., 13421,  9188, 18643],
        ...,
        [ 9085,   739,   466,  ..., 20572,  9136,  9320],
        [10595,   739,   466,  ...,  9222,  8098,  9185],
        [ 9085,   739,   466,  ..., 22334, 10799, 28300]])
Tokenized: torch.Size([807, 128]) - tensor([[ 9045,   739,   466,  ..., 19561, 12346,   387],
        [ 9130,   739,   466,  ...,  9566,  9181,  6919],
        [ 9130,   739,   466,  ..., 13421,  9188, 18643],
        ...,
        [ 9085,   739,   466,  ..., 20572,  9136,  9320],
        [10595,   739,   466,  ...,  9222,  8098,  9185],
        [ 9085,   739,   466,  ..., 22334, 10799, 28300]])
Tokenized: torch.Size([90, 128]) - tensor([[ 9143,   739,   466,  ...,  7047,  9131, 10765],
        [10595,   739,   466,  ..., 37999,  6905,  7965],
        [ 9085,   739,   466,  ..., 13969,   389, 121

In [21]:
torch.cuda.empty_cache()

In [22]:
from fastai.text.all import Learner, CrossEntropyLossFlat, Perplexity, Callback

# DataLoader와 모델 확인 후, Learner 초기화
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=Perplexity()).to_fp16()

In [23]:
learn.fine_tune(1)

epoch,train_loss,valid_loss,perplexity,time


OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Of the allocated memory 22.48 GiB is allocated by PyTorch, and 27.84 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)